In [ ]:
import molpy as mp
# import molvis as mv

class Methane(mp.Struct):

    def __init__(self):
        super().__init__()
        C = self.add_atom(name="C",)
        H1 = self.add_atom(name="H1", xyz=(-0.1, 0, -0.07))
        H2 = self.add_atom(name="H2", xyz=(0., 0.1, 0.07))
        H3 = self.add_atom(name="H3", xyz=(0., -0.1, 0.07))
        self.add_bond(C, H1)
        self.add_bond(C, H2)
        self.add_bond(C, H3)

methane = Methane()
mp.io.write_pdb("methane.pdb", methane)

TypeError: '<' not supported between instances of 'Methane' and 'Methane'